In [ ]:
!pip install evaluate
!pip install optuna
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 27.4 MB/s eta 0:00:00


In [ ]:
#Data handling & visualization
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Model evaluation<
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#PyTorch core
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
import torch.nn.utils.prune as prune
import torch.nn.functional as F

# Training utilities
from tqdm.auto import tqdm

# Transformers (Hugging Face)
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    RobertaTokenizer,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    get_scheduler,
    default_data_collator,
    AutoConfig

)


#Evaluation with HF Evaluate library
import evaluate

#Hyperparameter tuning
import optuna

from optuna import Trial

#Experiment tracking
import wandb

# File paths (optional)
from pathlib import Path

from datetime import datetime

from collections import Counter

import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import wandb
wandb.login(key="c76b3f4273b893fc55052d914d403d9718122ab7")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: maximilianon (maximilianon-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Code for Part B1: Fine-Tuning using the "full code" from exercise 4

In [ ]:
# Define model names
MODEL_NAME_TWITTER = "cardiffnlp/twitter-roberta-base"
MODEL_NAME_ELECTRA = "google/electra-base-discriminator"

# Load tokenizers
tokenizers = {
    MODEL_NAME_TWITTER: AutoTokenizer.from_pretrained(MODEL_NAME_TWITTER, use_fast=True),
    MODEL_NAME_ELECTRA: AutoTokenizer.from_pretrained(MODEL_NAME_ELECTRA)
}




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
# Label mapping
label2id = {
    "Extremely Negative": 0,
    "Negative": 1,
    "Neutral": 2,
    "Positive": 3,
    "Extremely Positive": 4
}

train_path = "/content/drive/MyDrive/Colab Notebooks/Corona_NLP_train_clean.csv"
test_path = "/content/drive/MyDrive/Colab Notebooks/Corona_NLP_test_clean.csv"

#Load the CSVs
train_df = pd.read_csv(train_path, encoding='ISO-8859-1')
test_df = pd.read_csv(test_path, encoding='ISO-8859-1')



#train_df = train_df.sample(n=50, random_state=42).reset_index(drop=True)


# Map sentiment labels to integers
train_df["label"] = train_df["Sentiment"].map(label2id)
test_df["label"] = test_df["Sentiment"].map(label2id)

train_df = train_df[["cleaned_tweets", "label"]]
test_df = test_df[["cleaned_tweets", "label"]]


for var in ["train_df", "test_df"]:
    df = locals()[var]
    df = df.dropna(subset=["cleaned_tweets"])                             # drop NaNs
    df = df[df["cleaned_tweets"].astype(str).str.strip() != ""].copy()    # drop empty/whitespace
    df = df[df["cleaned_tweets"].str.len() >= 3].reset_index(drop=True)   # optional: drop very short
    locals()[var] = df

# --- Normalize tweets (URLs, @users, numbers) ---
def normalize_tweet(t):
    t = str(t)
    t = re.sub(r"http\S+", "<url>", t)
    t = re.sub(r"@\w+", "<user>", t)
    t = re.sub(r"\d+", "<number>", t)
    return t.strip()

# Apply normalization after cleaning, before the split
train_df["cleaned_tweets"] = train_df["cleaned_tweets"].map(normalize_tweet)
test_df["cleaned_tweets"]  = test_df["cleaned_tweets"].map(normalize_tweet)


train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df["label"], random_state=42)

test_df_final = test_df.copy()

In [ ]:
# Define dataset
class TweetDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.texts = dataframe["cleaned_tweets"].tolist()
        self.labels = dataframe["label"].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # If batched access, HuggingFace might pass a list of indices
        if isinstance(idx, list):
            return [self._encode_single(i) for i in idx]
        return self._encode_single(idx)

    def _encode_single(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"][0],
            "attention_mask": encoding["attention_mask"][0],
            "labels": torch.tensor(label, dtype=torch.long)
        }





In [ ]:
def early_stop_check(patience, best_val_accuracy, best_val_accuracy_epoch,
                     current_val_accuracy, current_val_accuracy_epoch, min_delta=1e-4):
    early_stop_flag = False
    if current_val_accuracy > best_val_accuracy + min_delta:
        best_val_accuracy = current_val_accuracy
        best_val_accuracy_epoch = current_val_accuracy_epoch
    else:
        if current_val_accuracy_epoch - best_val_accuracy_epoch >= patience:
            early_stop_flag = True
    return best_val_accuracy, best_val_accuracy_epoch, early_stop_flag


In [ ]:
def train_model_with_hyperparams(model, train_loader, val_loader,optimizer, criterion, epochs, patience, trial,warmup_ratio=0.06, scheduler_name="linear"):
    best_val_accuracy = 0.0
    best_val_accuracy_epoch = 0
    best_model_state = None

    model = model.to(device)

    # Scheduler with warmup
    total_steps  = len(train_loader) * epochs
    warmup_steps = int(warmup_ratio * total_steps)
    lr_scheduler = get_scheduler(
        name=scheduler_name,
        optimizer=optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0.0
        total_train = 0
        correct_train = 0

        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # stability
            optimizer.step()
            lr_scheduler.step()

            bs = input_ids.size(0)
            train_loss += loss.item() * bs
            total_train += bs
            correct_train += (logits.argmax(dim=1) == labels).sum().item()

        train_loss /= max(total_train, 1)
        train_acc = correct_train / max(total_train, 1)

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        total_val = 0
        correct_val = 0
        all_val_labels, all_val_preds = [], []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                loss = criterion(logits, labels)

                bs = input_ids.size(0)
                val_loss += loss.item() * bs
                total_val += bs
                preds = logits.argmax(dim=1)

                correct_val += (preds == labels).sum().item()
                all_val_labels.extend(labels.cpu().numpy())
                all_val_preds.extend(preds.cpu().numpy())

        val_loss /= max(total_val, 1)
        val_acc = correct_val / max(total_val, 1)

        # Metrics
        val_precision = precision_score(all_val_labels, all_val_preds, average='weighted', zero_division=0)
        val_recall    = recall_score(all_val_labels,  all_val_preds, average='weighted', zero_division=0)
        val_f1_w      = f1_score(all_val_labels,      all_val_preds, average='weighted', zero_division=0)
        val_f1_macro  = f1_score(all_val_labels,      all_val_preds, average='macro',    zero_division=0)

        wandb.log({
            "Epoch": epoch,
            "Train Loss": train_loss,
            "Train Accuracy": train_acc,
            "Validation Loss": val_loss,
            "Validation Accuracy": val_acc,
            "Validation Precision": val_precision,
            "Validation Recall": val_recall,
            "Validation F1 Weighted": val_f1_w,
            "Validation F1 Macro": val_f1_macro,
            "Current LR": optimizer.param_groups[0]['lr'],
        })

        # --- Early stopping via your helper ---
        prev_best = best_val_accuracy
        best_val_accuracy, best_val_accuracy_epoch, early_stop_flag = early_stop_check(
            patience=patience,
            best_val_accuracy=best_val_accuracy,
            best_val_accuracy_epoch=best_val_accuracy_epoch,
            current_val_accuracy=val_acc,
            current_val_accuracy_epoch=epoch,
            min_delta=1e-4,   # tweak if you want stricter/looser improvement
        )

        # Save best model only on real improvement
        if best_val_accuracy > prev_best:
            best_model_state = {k: v.cpu() for k, v in model.state_dict().items()}

        if early_stop_flag:
            break

        # Optuna pruning
        if trial is not None:
            trial.report(val_acc, step=epoch)
            if trial.should_prune():
                raise optuna.TrialPruned()

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    return best_val_accuracy



In [ ]:
def objective(trial, model_name,tokenizer):
    # --- HP space (unchanged) ---
    learning_rate = trial.suggest_float("learning_rate", 8e-6, 8e-5, log=True)
    max_length    = trial.suggest_categorical("max_length", [128, 192, 256])
    batch_size    = trial.suggest_categorical("batch_size", [16,32])
    patience      = trial.suggest_int("patience", 2,3)
    epochs        = trial.suggest_int("epochs", 6, 8) # Added high value for epochs
    weight_decay  = trial.suggest_categorical("weight_decay", [0.0, 0.01, 0.02, 0.05])  # Include no-reg + wider range
    warmup_ratio  = trial.suggest_categorical("warmup_ratio", [0.0, 0.06, 0.1, 0.15])
    dropout = trial.suggest_float("dropout", 0.1, 0.25)
    scheduler     = "linear"


    # --- Fresh tokenizer per trial ---
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    tokenizer.add_special_tokens({"additional_special_tokens": ["<url>", "<user>", "<number>"]})

    # --- Data (tokenize with this tokenizer) ---
    train_dataset = TweetDataset(train_df, tokenizer, max_length=max_length)
    val_dataset   = TweetDataset(val_df,   tokenizer, max_length=max_length)
    train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    val_loader    = DataLoader(val_dataset,   batch_size=batch_size,               pin_memory=True)

      # --- Model ---
    config = AutoConfig.from_pretrained(model_name, num_labels=5, hidden_dropout_prob=dropout, attention_probs_dropout_prob=dropout)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

    # Always resize to tokenizer length (safe even if no tokens were added)
    model.resize_token_embeddings(len(tokenizer))

    # --- Class-weighted loss ---
    import numpy as np
    counts = np.bincount(train_df["label"].values, minlength=5)
    weights = 1.0 / np.maximum(counts, 1)
    class_weights = torch.tensor(weights, dtype=torch.float, device=device)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # --- Discriminative LRs (4x head) ---
    head_params, backbone_params = [], []
    for n, p in model.named_parameters():
        (head_params if "classifier" in n else backbone_params).append(p)

    base_lr = learning_rate
    head_lr = base_lr * 4.0
    optimizer = AdamW(
        [
            {"params": backbone_params, "lr": base_lr, "weight_decay": weight_decay},
            {"params": head_params,     "lr": head_lr,  "weight_decay": 0.0},
        ]
    )

    # --- W&B (sanitize slashes) ---
    safe_model_name = model_name.replace("/", "-")
    wandb.init(
        project=f"{safe_model_name}-fullcode-attempt2",
        name=f"trial-{trial.number}",
        reinit=True,
        config=dict(
            learning_rate=learning_rate, head_lr=head_lr, batch_size=batch_size,
            patience=patience, max_length=max_length,
            weight_decay=weight_decay, scheduler=scheduler, warmup_ratio=warmup_ratio,epochs=epochs,dropout=dropout,

        ),
    )

    try:
        val_accuracy = train_model_with_hyperparams(
            model, train_loader, val_loader, optimizer, criterion,
            epochs, patience, trial, warmup_ratio=warmup_ratio, scheduler_name=scheduler
        )
        return val_accuracy
    finally:
        # Clean up GPU even on prune/error
        try:
            wandb.finish()
        except Exception:
            pass
        model.to("cpu"); del model, optimizer
        import gc; gc.collect()
        torch.cuda.empty_cache()

In [ ]:
# RoBERTa
study = optuna.create_study(direction="maximize")
study.optimize(lambda trial: objective(trial, model_name=MODEL_NAME_TWITTER, tokenizer=tokenizers[MODEL_NAME_TWITTER]), n_trials=10)

# Electra
#study_electra = optuna.create_study(direction="maximize")
#study_electra.optimize(lambda trial: objective(trial, model_name=MODEL_NAME_ELECTRA, tokenizer=tokenizers[MODEL_NAME_ELECTRA]), n_trials=10)


[I 2025-08-17 10:04:24,967] A new study created in memory with name: no-name-df00a597-94d0-424a-bff7-b0bb88cb521f


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


...

[I 2025-08-17 12:23:56,969] Trial 7 pruned. 
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Current LR,▁
Epoch,▁
Train Accuracy,▁
Train Loss,▁
Validation Accuracy,▁
Validation F1 Macro,▁
Validation F1 Weighted,▁
Validation Loss,▁
Validation Precision,▁
Validation Recall,▁
Current LR,3e-05


[I 2025-08-17 12:27:15,539] Trial 8 pruned. 
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Current LR,█▁
Epoch,▁█
Train Accuracy,▁█
Train Loss,█▁
Validation Accuracy,█▁
Validation F1 Macro,█▁
Validation F1 Weighted,█▁
Validation Loss,▁█
Validation Precision,▁█
Validation Recall,█▁
Current LR,4e-05


[I 2025-08-17 12:38:14,980] Trial 9 pruned. 


REST OF THINGS

In [ ]:
# === Best HPs from W&B (trial-0) ===
BEST = {
    "batch_size":   16,
    "dropout":      0.18220476189798832,
    "epochs":       6,
    "head_lr":      2.091e-4,        # 0.0002091
    "learning_rate":5.22751e-05,
    "max_length":   192,
    "patience":     2,
    "scheduler":    "linear",
    "warmup_ratio": 0.06,
    "weight_decay": 0.01,
}

MODEL_NAME = "cardiffnlp/twitter-roberta-base"   # change if needed
SAVE_DIR   = "/content/drive/MyDrive/Colab Notebooks/best_roberta_trial0"

from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import AdamW
import numpy as np
import wandb, os, torch

# Init W&B run (train_model_with_hyperparams uses wandb.log)
wandb.init(
    project="twitter-roberta-fullcode",
    name="best_retrain_trial-0",
    config=BEST,
    reinit=True
)

# --- Tokenizer with your special tokens ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.add_special_tokens({"additional_special_tokens": ["<url>", "<user>", "<number>"]})

# --- Data loaders with best max_length / batch_size ---
train_dataset = TweetDataset(train_df, tokenizer, max_length=BEST["max_length"])
val_dataset   = TweetDataset(val_df,   tokenizer, max_length=BEST["max_length"])

train_loader  = DataLoader(train_dataset, batch_size=BEST["batch_size"], shuffle=True, pin_memory=True)
val_loader    = DataLoader(val_dataset,   batch_size=BEST["batch_size"],               pin_memory=True)

# --- Model with best dropout ---
config = AutoConfig.from_pretrained(
    MODEL_NAME,
    num_labels=5,
    hidden_dropout_prob=BEST["dropout"],
    attention_probs_dropout_prob=BEST["dropout"],
    id2label={i: k for k, i in label2id.items()},
    label2id=label2id
)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config)
model.resize_token_embeddings(len(tokenizer))
model = model.to(device)

# --- Class-weighted loss (same recipe you used) ---
counts = np.bincount(train_df["label"].values, minlength=5)
weights = 1.0 / np.maximum(counts, 1)
class_weights = torch.tensor(weights, dtype=torch.float, device=device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# --- Discriminative LRs: backbone vs head ---
head_params, backbone_params = [], []
for n, p in model.named_parameters():
    (head_params if "classifier" in n else backbone_params).append(p)

optimizer = AdamW(
    [
        {"params": backbone_params, "lr": BEST["learning_rate"], "weight_decay": BEST["weight_decay"]},
        {"params": head_params,     "lr": BEST["head_lr"],       "weight_decay": 0.0},
    ]
)

# --- Train with your helper (does early stop + best state restore) ---
best_val_acc = train_model_with_hyperparams(
    model, train_loader, val_loader, optimizer, criterion,
    epochs=BEST["epochs"], patience=BEST["patience"], trial=None,
    warmup_ratio=BEST["warmup_ratio"], scheduler_name=BEST["scheduler"]
)
print(f"[trial-0] Rebuilt best model. Val accuracy = {best_val_acc:.4f}")

# --- Save persistently ---
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_pretrained(SAVE_DIR)
tokenizer.save_pretrained(SAVE_DIR)
print("Saved best model and tokenizer to:", SAVE_DIR)

wandb.finish()


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


[trial-0] Rebuilt best model. Val accuracy = 0.7099
Saved best model and tokenizer to: /content/drive/MyDrive/Colab Notebooks/best_roberta_trial0


Current LR,█▇▅▄▂▁
Epoch,▁▂▄▅▇█
Train Accuracy,▁▄▆▆▇█
Train Loss,█▅▄▃▂▁
Validation Accuracy,▁▅▆▇██
Validation F1 Macro,▁▄▆▇██
Validation F1 Weighted,▁▄▆▇██
Validation Loss,█▄▁▁▃▃
Validation Precision,▁▅▆▇██
Validation Recall,▁▅▆▇██
Current LR,0


Compression techniques

In [ ]:
import os, torch, copy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch import nn
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Point to your saved checkpoint folder (where config.json + pytorch_model.bin live)
BASE_DIR = "/content/drive/MyDrive/Colab Notebooks/best_roberta_trial0"  # or your local path

model = AutoModelForSequenceClassification.from_pretrained(BASE_DIR).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(BASE_DIR)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

def evaluate_on(df, model, tokenizer, max_length=192, batch_size=64, device=DEVICE, desc="Eval"):
    class TweetDataset(torch.utils.data.Dataset):
        def __init__(self, dataframe, tokenizer, max_length=128):
            self.texts = dataframe["cleaned_tweets"].tolist()
            self.labels = dataframe["label"].tolist()
            self.tokenizer = tokenizer
            self.max_length = max_length
        def __len__(self): return len(self.texts)
        def __getitem__(self, idx):
            enc = self.tokenizer(
                str(self.texts[idx]),
                truncation=True,
                padding="max_length",
                max_length=self.max_length,
                return_tensors="pt",
            )
            return {
                "input_ids": enc["input_ids"][0],
                "attention_mask": enc["attention_mask"][0],
                "labels": torch.tensor(self.labels[idx], dtype=torch.long),
            }

    ds = TweetDataset(test_df_final, tokenizer, max_length=max_length)
    dl = DataLoader(ds, batch_size=batch_size, pin_memory=True)

    model = model.to(device).eval()
    all_y, all_p = [], []
    with torch.no_grad():
        for b in dl:
            ids = b["input_ids"].to(device)
            att = b["attention_mask"].to(device)
            y   = b["labels"].to(device)
            logits = model(ids, attention_mask=att).logits
            p = logits.argmax(dim=1)
            all_y.extend(y.cpu().numpy().tolist())
            all_p.extend(p.cpu().numpy().tolist())

    print(f"{desc} accuracy:", accuracy_score(all_y, all_p))
    print(classification_report(all_y, all_p, digits=4))


In [ ]:
def compress_model(base_model):
    compressed = {}

    # 1) Dynamic Quantization (CPU-only module)
    cpu_model = copy.deepcopy(base_model).to("cpu")
    qmodel = torch.quantization.quantize_dynamic(
        cpu_model,
        {nn.Linear},
        dtype=torch.qint8
    )
    compressed["quantized_cpu"] = qmodel  # keep in memory; will save via state_dict

    # 2) Pruning (unstructured L1 across Linear layers)
    pruned = copy.deepcopy(base_model).to(DEVICE)
    params_to_prune = []
    for m in pruned.modules():
        if isinstance(m, nn.Linear):
            params_to_prune.append((m, "weight"))

    if len(params_to_prune) > 0:
        prune.global_unstructured(
            params_to_prune,
            pruning_method=prune.L1Unstructured,
            amount=0.40,  # 40% sparsity
        )
        # IMPORTANT: remove reparam to bake weights into .weight
        for (m, _) in params_to_prune:
            try:
                prune.remove(m, "weight")
            except Exception:
                pass
    compressed["pruned"] = pruned

    # 3) FP16 (good for GPU inference)
    half_model = copy.deepcopy(base_model).half().to(DEVICE)
    compressed["fp16"] = half_model

    return compressed

compressed_models = compress_model(model)


In [ ]:
# Base FP32
evaluate_on(test_df_final, model, tokenizer, max_length=192, desc="Base FP32")

# FP16 (GPU)
evaluate_on(test_df_final, compressed_models["fp16"], tokenizer, max_length=192, desc="FP16")

# Pruned (GPU)
evaluate_on(test_df_final, compressed_models["pruned"], tokenizer, max_length=192, desc="Pruned")

# Quantized (CPU)
evaluate_on(test_df_final, compressed_models["quantized_cpu"], tokenizer, max_length=192, device="cpu", desc="Quantized CPU")


Base FP32 accuracy: 0.6793993677555321
              precision    recall  f1-score   support

           0     0.6585    0.8176    0.7295       592
           1     0.6262    0.6100    0.6180      1041
           2     0.8030    0.6872    0.7406       617
           3     0.6675    0.5892    0.6259       947
           4     0.6999    0.7980    0.7457       599

    accuracy                         0.6794      3796
   macro avg     0.6910    0.7004    0.6919      3796
weighted avg     0.6819    0.6794    0.6774      3796

FP16 accuracy: 0.6791359325605901
              precision    recall  f1-score   support

           0     0.6585    0.8176    0.7295       592
           1     0.6262    0.6100    0.6180      1041
           2     0.8015    0.6872    0.7400       617
           3     0.6671    0.5882    0.6251       947
           4     0.6999    0.7980    0.7457       599

    accuracy                         0.6791      3796
   macro avg     0.6906    0.7002    0.6917      3796
weig

In [ ]:
OUT_DIR = "/content/drive/MyDrive/Colab Notebooks/best_roberta_trial0_compressed"
os.makedirs(OUT_DIR, exist_ok=True)

# A) Save FP16
fp16_dir = os.path.join(OUT_DIR, "fp16")
os.makedirs(fp16_dir, exist_ok=True)
compressed_models["fp16"].save_pretrained(fp16_dir)
tokenizer.save_pretrained(fp16_dir)

# B) Save pruned (after prune.remove, save_pretrained works)
pruned_dir = os.path.join(OUT_DIR, "pruned")
os.makedirs(pruned_dir, exist_ok=True)
compressed_models["pruned"].save_pretrained(pruned_dir)
tokenizer.save_pretrained(pruned_dir)

# C) Save quantized CPU (state_dict + a tiny loader script)
q_dir = os.path.join(OUT_DIR, "quantized_cpu")
os.makedirs(q_dir, exist_ok=True)
torch.save(compressed_models["quantized_cpu"].state_dict(), os.path.join(q_dir, "quantized_state_dict.pt"))

# Save a small loader so future-you can reload easily
with open(os.path.join(q_dir, "load_quantized.py"), "w") as f:
    f.write(
        "import torch\n"
        "from torch import nn\n"
        "from transformers import AutoModelForSequenceClassification\n"
        "def load_quantized(model_dir, state_path):\n"
        "    model = AutoModelForSequenceClassification.from_pretrained(model_dir)\n"
        "    model = torch.quantization.quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)\n"
        "    sd = torch.load(state_path, map_location='cpu')\n"
        "    model.load_state_dict(sd, strict=False)\n"
        "    model.eval()\n"
        "    return model\n"
    )

print("Saved to:", OUT_DIR)


Saved to: /content/drive/MyDrive/Colab Notebooks/best_roberta_trial0_compressed
